In [3]:
import sys,os
__script_path=os.path.abspath(globals().get('__file__','.'))
__script_dir = os.path.dirname(__script_path)
root_dir = os.path.abspath(f'{__script_dir}/..')
print(root_dir)
for lib in [root_dir][::-1]:
    if lib in sys.path:
        sys.path.remove(lib)
    sys.path.insert(0,lib)


c:\Users\Admin\Data\multimodal-rag-baseline


In [11]:
from configs.config import *
from libs.common import *
from utils.mongodb_connection import *
from utils.rag_utils import *
from utils.format_utils import *
load_dotenv(find_dotenv())

True

In [ ]:
mongo_uri = os.getenv('MONGO_URI')
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
collection = "kyanon_digital"

mongo_client = get_mongo_client(mongo_uri)
db = mongo_client['Kyanon']
collection = db['RAG']

In [ ]:
# https://huggingface.co/thenlper/gte-large
embedding_model = SentenceTransformer("thenlper/gte-large")
embedding_model = embedding_model.to('cuda')

print("Model is using GPU:", next(embedding_model.parameters()).is_cuda)

In [ ]:
import camelot

pdf_path = "content.pdf"

tables = camelot.read_pdf(pdf_path, pages="all", flavor='lattice')
print(f"Tổng cộng: {tables.n} bảng được tìm thấy.")

table_strings = []

for i, table in enumerate(tables):
    table_str = table.df.to_markdown(index=False)  # Dễ đọc hơn với index=False
    table_strings.append(table_str)
    print(f"\n--- Bảng {i + 1} ---")
    print(table_str)



c:\Users\Admin\miniconda3\envs\AI\lib\site-packages\pypdf\_crypt_providers\_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from this module in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4


Tổng cộng: 10 bảng được tìm thấy.

--- Bảng 1 ---
| 0   | 1                                                 |
|:----|:--------------------------------------------------|
|     | Elected to the Hall of Fame on this ballot (named |
|     | in bold italics).                                 |
|     | Elected subsequently, as of 2025 (named in plain  |
|     | italics).                                         |
|     | Renominated for the 2019 BBWAA election by        |
|     | adequate performance on this ballot and has not   |
|     | subsequently been eliminated.                     |
|     | Eliminated from annual BBWAA consideration by     |
|     | poor performance or expiration on subsequent      |

--- Bảng 2 ---
| 0   | 1                                                 |
|:----|:--------------------------------------------------|
|     | ballots.                                          |
|     | Eliminated from annual BBWAA consideration by     |
|     | poor performance or expira

In [ ]:
import requests
import json

def generate_response(prompt):
    """
    Gửi prompt đến Ollama và nhận về một đoạn phản hồi hoàn chỉnh.
    
    Tham số:
        prompt (str): Câu hỏi hoặc yêu cầu bạn muốn gửi đến mô hình.

    Trả về:
        str: Phản hồi hoàn chỉnh từ mô hình.
    """
    full_response = ""
    context = f"""
   You are given a list of table fragments extracted from a PDF document using OCR. Some tables were split across multiple pages and therefore appear as separate fragments in the input.

    TASK INSTRUCTIONS:
    Do NOT merge any tables.
    For each table fragment , provide a clear and concise summary that includes:
    Number of columns
    Presence and content of headers (if any)
    General data pattern/type in each column (e.g., numeric, text, date, mixed)
    Any notable features such as empty cells, repeated rows, formatting inconsistencies, etc.
    If two or more fragments have identical structures , indicate this in the summaries.
    Preserve original formatting — do not modify or reorder any data.
    Please output:

    A summary for each table fragment, analyze and provide a summary for each table fragment , then merge fragments with identical summaries into full tables. Return only the final merged tables and unmerged fragments in markdown format.
    {prompt}
    """
    url = "http://localhost:11434/api/generate"
    payload = {
        "model": "gemma3:latest",
        "prompt": context,
        "stream": True
    }

    with requests.post(url, json=payload, stream=True) as response:
        for line in response.iter_lines():
            if line:
                try:
                    data = json.loads(line)
                    if 'response' in data:
                        chunk = data['response']
                        full_response += chunk
                    if data.get('done', False):
                        break
                except json.JSONDecodeError:
                    continue  

    return full_response.strip()

In [ ]:
for i in tables:
    table_summary = generate_response(i.df.to_markdown(index=False))
    embedding = get_embedding(table_summary, embedding_model)
    
    print(f"\n--- Tóm tắt bảng {i + 1} ---")
    chunk_tables = []
    
    document = {
        "embedding": embedding,
        "content": {
            "table": i.df.to_markdown(index=False),
            "summary": table_summary
        }
            
    }
    collection.insert_one(document)

In [ ]:
all_tables_combined = ""

for i, table in enumerate(tables):
    table_str = table.df.to_markdown(index=False)  
    all_tables_combined += f"\n\n--- Bảng {i + 1} ---\n"
    all_tables_combined += table_str



print("\n--- Toàn bộ bảng đã gộp ---")
print(all_tables_combined)

In [ ]:
query = "What is The BBWAA election rules?"
source_information = get_search_result(embedding_model, query.lower(), collection)
source_information

In [ ]:
combined_information = f"Become an expert on the concepts on Wikipedia. Customer's question: {query}\nAnswer the question based on the following document information: {source_information}."
print('==== LLMs Inference')
print(combined_information)

In [ ]:
from huggingface_hub import login

api_token = os.getenv('HUGGINGE_FACE_TOKEN') 
login(api_token)



In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it", device_map="auto")

In [ ]:
input_ids = tokenizer(combined_information, return_tensors="pt").to("cuda")
response = model.generate(**input_ids, max_new_tokens=100)
print(tokenizer.decode(response[0]))